In [1]:
!pip install chromadb openai 

In [1]:
import chromadb
from chromadb.config import Settings
from chromadb.utils import embedding_functions


# client = chromadb.Client(Settings(chroma_db_impl="duckdb+parquet",
#                                     persist_directory="db/"
#                                 ))

client = chromadb.PersistentClient(path="db")

code sets up a client to interact with ChromaDB Database
directory for stopring database files db/


In [2]:
embedding_fn = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

/Users/jacobschorr/Documents/RandyCode/chromaDB/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
collection = client.get_or_create_collection(name="Courses", embedding_function=embedding_fn)

In [18]:
computer_science = """
A Computer Science major focuses on the study of algorithms, programming, software development, and computational theory. Students learn various programming languages, data structures, and system architectures to develop efficient and scalable applications. The major also covers areas like artificial intelligence, cybersecurity, databases, and networking, preparing students for diverse tech careers. Hands-on projects and internships help build problem-solving skills and real-world experience in software engineering, machine learning, or web development. Graduates can pursue roles such as software developer, data scientist, or IT specialist in industries ranging from finance to healthcare and technology.
"""

history= """
A History major explores past events, societies, cultures, and significant figures to understand how they shape the present and future. Students analyze primary and secondary sources, learning to think critically, interpret evidence, and construct well-supported arguments. The major covers various periods and regions, from ancient civilizations to modern global conflicts, often incorporating political, economic, and social perspectives. Research projects and discussions help develop strong writing, analytical, and communication skills. Graduates can pursue careers in education, law, public policy, museum curation, or journalism, leveraging their ability to synthesize complex information and contextualize historical trends.

"""

In [4]:
import csv
#DONT NEED TO RERUN
def read_csv_to_list(file_path):
    documents = []
    metadata = []
    ids = []
    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.reader(csvfile)
        for i, row in enumerate(reader):
            documents.append(', '.join(row))  # Convert row to a string
            if len(row) > 1:
                course_code = row[1].split(' -')[0]  # Strip everything including and after '-'
                metadata.append({'course_code': course_code})  # Extract cleaned course code as metadata
            ids.append(f"doc_{i}")  # Assign a unique ID to each document
    return documents, metadata, ids

# Example usage
file_path = 'subject_courses.csv'  # Replace with your actual file path
course_documents, course_metadata, ids = read_csv_to_list(file_path)



In [5]:
#DONT NEED TO RERUN
collection.upsert(documents=course_documents,
metadatas = course_metadata,
ids = ids,
)

KeyboardInterrupt: 

In [ ]:
#DONT NEED TO RERUN
import pdfplumber
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json
import uuid

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file and return a list of texts per page."""
    text_by_page = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text = page.extract_text() or ""  # Extract text or set empty string if none
            text_by_page.append(text)
    return text_by_page

def chunk_text(text_list, chunk_size=1000, chunk_overlap=200):
    """Split text into overlapping chunks and store metadata for ChromaDB."""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""]
    )

    documents = []
    metadatas = []
    ids = []

    for i, page_text in enumerate(text_list):
        chunks = text_splitter.split_text(page_text)
        for chunk in chunks:
            chunk_id = str(uuid.uuid4())  # Generate unique ID
            documents.append(chunk)
            metadatas.append({"page": i + 1})  # Store page number as metadata
            ids.append(chunk_id)
    
    return documents, metadatas, ids

def save_chunks_to_json(documents, metadatas, ids, output_file="chunks.json"):
    """Save extracted chunks to a JSON file."""
    chunk_data = [
        {"id": ids[i], "text": documents[i], "metadata": metadatas[i]}
        for i in range(len(documents))
    ]
    
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(chunk_data, f, indent=4)

# Example Usage
pdf_path = "Undergraduate_Catalog_shortenend.pdf"  # Update with actual PDF file
text_list = extract_text_from_pdf(pdf_path)
documents, metadatas, ids = chunk_text(text_list)

# Save chunks for reference or direct insertion into ChromaDB
save_chunks_to_json(documents, metadatas, ids)
print(f"Extracted {len(documents)} chunks. Saved to chunks.json.")


In [ ]:
catalog_documents = documents
catalog_metadatas = metadatas
catalog_ids = ids


In [19]:
#DONT NEED TO RERUN
collection = client.get_or_create_collection(name="undergraduate_catalog")

# Load JSON and insert into ChromaDB
import json
with open("chunks.json", "r", encoding="utf-8") as f:
    chunk_data = json.load(f)

# Insert into ChromaDB
collection.upsert(
    ids=[chunk["id"] for chunk in chunk_data],
    documents=[chunk["text"] for chunk in chunk_data],
    metadatas=[chunk["metadata"] for chunk in chunk_data]
)


collection object is similar to creating a table in a traditional database
**Collection** is a logical grouping of data that share a context or purpose (one major in our case):
A collection contains
1. documents (text or data that you are storing)
2. metadata (slourse, date, tags etc)
3. Embeddings
4. IDs

In [6]:
print(collection.count())

# private endpoint if running http server (in that case, its fine):
print(client.list_collections())

3928
['Courses', 'Majors', 'undergraduate_catalog']


collection object is similar to creating a table in a traditional database
**Collection** is a logical grouping of data that share a context or purpose (one major in our case):
A collection contains
1. documents (text or data that you are storing)
2. metadata (slourse, date, tags etc)
3. Embeddings
4. IDs

In [ ]:
collection = client.get_or_create_collection(name="Courses", embedding_function=embedding_fn)
results = collection.query(
    query_texts=["what requirements are there for computer science"], # Chroma will embed this for you
    n_results=8, # how many results to return
    include=[ 'documents','metadatas']
)
print(results)

In [8]:
collection = client.get_or_create_collection(name="undergraduate_catalog")

In [ ]:
results = collection.query(
    query_texts=["what requirements are there for computer science"], # Chroma will embed this for you
    n_results=8, # how many results to return
    include=[ 'documents','metadatas']
)
print(results)